In [76]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [77]:
data_path = 'marketing_campaign.csv'

campaign = pd.read_csv(data_path)

In [78]:
campaign.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,...,88,3,8,10,4,7,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,...,6,2,1,1,2,5,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,...,42,1,8,2,10,4,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,...,5,2,2,0,4,6,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,...,15,5,5,3,6,5,0,3,11,0


In [79]:
dummy_fields = ['Year_Birth', 'Education', 'Marital_Status']
for each in dummy_fields:
    dummies = pd.get_dummies(campaign[each], prefix=each, drop_first=False)
    campaign = pd.concat([campaign, dummies], axis=1)

fields_to_drop = ['ID', 'Dt_Customer', 'Year_Birth', 'Education', 'Marital_Status']
data = campaign.drop(fields_to_drop, axis=1)
data.head()


,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,...,Education_Master,Education_PhD,Marital_Status_Absurd,Marital_Status_Alone,Marital_Status_Divorced,Marital_Status_Married,Marital_Status_Single,Marital_Status_Together,Marital_Status_Widow,Marital_Status_YOLO
0,58138.0,0,0,58,635,88,546,172,88,88,...,0,0,0,0,0,0,1,0,0,0
1,46344.0,1,1,38,11,1,6,2,1,6,...,0,0,0,0,0,0,1,0,0,0
2,71613.0,0,0,26,426,49,127,111,21,42,...,0,0,0,0,0,0,0,1,0,0
3,26646.0,1,0,26,11,4,20,10,3,5,...,0,0,0,0,0,0,0,1,0,0
4,58293.0,1,0,94,173,43,118,46,27,15,...,0,1,0,0,0,1,0,0,0,0


In [80]:
print(data.columns.values)

['Income' 'Kidhome' 'Teenhome' 'Recency' 'MntWines' 'MntFruits'
 'MntMeatProducts' 'MntFishProducts' 'MntSweetProducts' 'MntGoldProds'
 'NumDealsPurchases' 'NumWebPurchases' 'NumCatalogPurchases'
 'NumStorePurchases' 'NumWebVisitsMonth' 'Complain' 'Z_CostContact'
 'Z_Revenue' 'Response' 'Year_Birth_1893' 'Year_Birth_1899'
 'Year_Birth_1900' 'Year_Birth_1940' 'Year_Birth_1941' 'Year_Birth_1943'
 'Year_Birth_1944' 'Year_Birth_1945' 'Year_Birth_1946' 'Year_Birth_1947'
 'Year_Birth_1948' 'Year_Birth_1949' 'Year_Birth_1950' 'Year_Birth_1951'
 'Year_Birth_1952' 'Year_Birth_1953' 'Year_Birth_1954' 'Year_Birth_1955'
 'Year_Birth_1956' 'Year_Birth_1957' 'Year_Birth_1958' 'Year_Birth_1959'
 'Year_Birth_1960' 'Year_Birth_1961' 'Year_Birth_1962' 'Year_Birth_1963'
 'Year_Birth_1964' 'Year_Birth_1965' 'Year_Birth_1966' 'Year_Birth_1967'
 'Year_Birth_1968' 'Year_Birth_1969' 'Year_Birth_1970' 'Year_Birth_1971'
 'Year_Birth_1972' 'Year_Birth_1973' 'Year_Birth_1974' 'Year_Birth_1975'
 'Year_Birth_1976' 

### Scaling target variables

In [81]:
quant_features = ['Income', 'Recency', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts','MntSweetProducts','MntGoldProds','NumDealsPurchases','NumWebPurchases','NumCatalogPurchases','NumStorePurchases','NumWebVisitsMonth','Complain','Z_CostContact','Z_Revenue']
# Store scalings in a dictionary so we can convert back later
scaled_features = {}
for each in quant_features:
    mean, std = data[each].mean(), data[each].std()
    scaled_features[each] = [mean, std]
    data.loc[:, each] = (data[each] - mean)/std


### training, validation and test data

In [82]:

test_data = data[-200:]

# Now remove the test data from the data set 
data = data[:-200]

# Separate the data into features and targets
target_fields = ['Response']
features, targets = data.drop(target_fields, axis=1), data[target_fields]
test_features, test_targets = test_data.drop(target_fields, axis=1), test_data[target_fields]

In [83]:
train_features, train_targets = features[:-300], targets[:-300]
val_features, val_targets = features[-300:], targets[-300:]

In [84]:
train_features=train_features.astype(np.float)
train_targets=train_targets.astype(np.float)
val_features=val_features.astype(np.float)
val_targets=val_targets.astype(np.float)

In [85]:
train_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1740 entries, 0 to 1739
Data columns (total 90 columns):
Income                     1723 non-null float64
Kidhome                    1740 non-null float64
Teenhome                   1740 non-null float64
Recency                    1740 non-null float64
MntWines                   1740 non-null float64
MntFruits                  1740 non-null float64
MntMeatProducts            1740 non-null float64
MntFishProducts            1740 non-null float64
MntSweetProducts           1740 non-null float64
MntGoldProds               1740 non-null float64
NumDealsPurchases          1740 non-null float64
NumWebPurchases            1740 non-null float64
NumCatalogPurchases        1740 non-null float64
NumStorePurchases          1740 non-null float64
NumWebVisitsMonth          1740 non-null float64
Complain                   1740 non-null float64
Z_CostContact              0 non-null float64
Z_Revenue                  0 non-null float64
Year_Birth_1893  

In [86]:
import torch

#Check if your system supports CUDA
gpu_available = torch.cuda.is_available() # is_available returns a bool as true or flase

if gpu_available:
    print('Training on GPU')
else:
    print('Training on CPU')

Training on GPU


In [87]:
train_features_t=torch.tensor(train_features.values)
train_targets_t = torch.tensor(train_targets.values)
val_features_t = torch.tensor(val_features.values)
val_targets_t = torch.tensor(val_targets.values)

In [88]:
train_features_t.size()

torch.Size([1740, 90])

In [89]:
train_targets_t.size()

torch.Size([1740, 1])

In [90]:
train_features_loader = torch.utils.data.DataLoader(train_features_t)
train_targets_loader = torch.utils.data.DataLoader(train_targets_t)
val_features_loader = torch.utils.data.DataLoader(val_features_t)
val_targets_loader = torch.utils.data.DataLoader(val_targets_t)

### Neural Network Architecture


In [102]:
import torch.nn as nn
import torch.nn.functional as F

# CNN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(90,128)
        self.fc2 = nn.Linear(128,64)
        self.fc3 = nn.Linear(64,1)
        
    def forward(self, x):
        ## Define forward behavior
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        return x

# instantiate the CNN
model = Net().double()
print(model)


Net(
  (fc1): Linear(in_features=90, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=1, bias=True)
)


In [103]:
import torch.optim as optim

criterion = nn.MSELoss()

# using Adam optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

### Training and Validation 1

In [104]:
# TODO: Train the network here
epochs = 5

for e in range(epochs):
    running_loss = 0
    for features, labels in zip(train_features_loader,train_targets_loader):              
        log_ps = model(features)
        loss = criterion(log_ps, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    else:
        print(f"Training loss: {running_loss/len(train_features_loader)}")

Training loss: nan
Training loss: nan
Training loss: nan
Training loss: nan
Training loss: nan
